# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
from nltk.stem import SnowballStemmer 

borrowers = pd.read_csv('/datasets/data.csv')


In [2]:
borrowers.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
borrowers.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [4]:
new_columns = ['children','days_employed', 'age', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'enployment_type', 'credit_debt', 'salary', 'purpose']
borrowers.set_axis(new_columns, axis = 'columns', inplace = True)

In [5]:
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
age                 21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
enployment_type     21525 non-null object
credit_debt         21525 non-null int64
salary              19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов:

- children — количество детей в семье (тип int64)
- days_employed — общий трудовой стаж в днях (тип float64)
- age — возраст клиента в годах (тип int64)
- education — уровень образования клиента (тип object)
- education_id — идентификатор уровня образования (тип int64)
- family_status — семейное положение (тип object)
- family_status_id — идентификатор семейного положения (тип int64)
- gender — пол клиента (тип object)
- enployment_type — тип занятости (тип object)
- credit_debt — имел ли задолженность по возврату кредитов (тип int64)
- salary — ежемесячный доход (тип float64)
- purpose — цель получения кредита (тип object)

Всего строк - 21524, но в столбцах "days_employed" и "salary" содержится по 19351 значений. Это говорит о том, что в данных есть пустые значения. Тип колонок "days_employed" и "salary" лучше изменить на целочисленный, в большей точности нет необходимости.

Каждая строка таблицы содержит данные о клиенте, описывая его семейное положение, количесво детей, уровень образования, место работы, уровень заработной платы, "кредитную историю" и цель взятия кредита. 

In [6]:
borrowers.describe()

,children,days_employed,age,education_id,family_status_id,credit_debt,salary
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В столбце "children" сразу видим минимальное и максимальное количество детей: -1 и 20. Вероятнее всего эти данные введены некорректно. Необходимо будет проверить сколько таких строк существует в таблице и обработать эти значения.

В столбце "days_employed" так же замечено несколько странных моментов:
1. Есть отрицательные значинения, что странно для обозначения трудового стажа в днях;
2. Максимальное значение - 401755 дней. Долго же кому-то пришлось работать, целых 1100 лет (а это еще и без выходных). Нужно будет обработать эти данные;
3. Среднее значение так же очень велико, это истекает из пункта выше. 

В столбце "age" минимальный возраст равен нулю. Скорее всего возраст просто не указали, нужно будет определить, сколько таких значений мы имеем, ведь для получения кредита это необходимые данные.

Также проверим, есть ли в нашем датафрейме клиенты младше 18-ти лет (помимо нулевых значений). Подсчитаем значения и проверим, совпадают ли эти строки со строками где возраст равен нулю.

In [7]:
borrowers[borrowers['age'] < 18]['age'].count()

101

In [8]:
borrowers[borrowers['age'] == 0]['age'].count()

101

Это одни и те же значения. В дальнейшем применим к ним категоризацию, исходя из типа занятости клиентов.

In [9]:
borrowers[borrowers['children']== -1]['children'].count()

47

Получили всего 47 значений. Cкорее всего кто-то ошибся, поставив туда "-". Возьмем весь столбец 'children' по модулю, чтобы избавиться от отрицательных значений. Сразу подсчитаем, количество значений по этому столбцу

In [10]:
borrowers['children'] = borrowers['children'].abs()
borrowers['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [11]:
borrowers[borrowers['days_employed']>0]['days_employed'].count()

3445

In [12]:
borrowers[borrowers['days_employed']>0]['days_employed'].min()

328728.72060451825

Самое минимальное значение оказалось не таким уж и маленьким. Проверим, есть ли зависимость этих больших значений от типа занятости клиента. Выведем уникальные типы занятости, где трудовой стаж больше нуля:

In [13]:
borrowers[borrowers['days_employed']>0]['enployment_type'].unique()

array(['пенсионер', 'безработный'], dtype=object)

Теперь подсчитаем сколько из общего количества положительных значений в столбце "days_employed" приходится на пенсионеров, а сколько - на безработных:

In [14]:
days_employed_of_unemployed = borrowers[(borrowers['days_employed']>0) & (borrowers['enployment_type'] == 'безработный')]['enployment_type'].count()
print('Количество положительных значений в столбце "days_employed" относящееся к безработным равно {}'.format(days_employed_of_unemployed))
days_employed_of_pensioner = borrowers[(borrowers['days_employed']>0) & (borrowers['enployment_type'] == 'пенсионер')]['enployment_type'].count()
print('Количество положительных значений в столбце "days_employed" относящееся к пенсионерам равно {}'.format(days_employed_of_pensioner))


Количество положительных значений в столбце "days_employed" относящееся к безработным равно 2
Количество положительных значений в столбце "days_employed" относящееся к пенсионерам равно 3443


Получается, что практически все эти огромные значения (от 900 лет трудового стажа) относятся к пенсионерам. Предположу, что в дальнейшем, если будет необходимо, при расчетах мы можем просто отталкиваться от средних показателей по этой выборке. Возможно эти значения такие большие, так как пенсионеры работали еще до того момента, когда появилось понятие "Unix-время" и их правильное преобразование просто не получилось выполнить.

Так же 2 значения попали в категорию к безработным. Проверим количество безработных в нашей таблице, чтобы в случае чего в дальнейшем заменить эти два больших значения на среднее между безработными.

In [15]:
 borrowers[borrowers['enployment_type'] == 'безработный']['enployment_type'].count()

2

Безработных в таблице всего двое. Проверим, вдруг по возрасту они подходят под категорию "пенсионеры":

In [16]:
 borrowers[borrowers['enployment_type'] == 'безработный']['age'].unique()

array([31, 45])

Исходя из возраста, эти люди - вряд ли пенсионеры. Эти два значения можем оставить, они не должны повлиять на наш дальнейший анализ.

### Обработка пропусков

In [17]:
borrowers.isnull().sum()

children               0
days_employed       2174
age                    0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
enployment_type        0
credit_debt            0
salary              2174
purpose                0
dtype: int64

In [18]:
len(borrowers[(borrowers['days_employed'].isnull() == True) & (borrowers['salary'].isnull() == True)])

2174

Соблюдая оба эти условия, мы получили 2174 строки - ровно столько, сколько пропусков мы имеем, то есть пустые значения по этим столбцам находятся в одних и тех же строках. Можно заполнить эти пропуски средним значением, исходя из типа занятости клиента. Поэтому подсчитаем количество строк с этими пропусками по столбцу "enployment_type". 

Подсчитаем какие типы занятости и в каком количестве относятся к данным условиям: 

In [19]:
borrowers[borrowers['days_employed'].isnull() == True]['enployment_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: enployment_type, dtype: int64

In [20]:
print("Количество отрицательных значений days_employed:",borrowers[borrowers['days_employed']<0].shape[0])
print("Количество положительных значений days_employed:",borrowers[borrowers['days_employed']>0].shape[0])


Количество отрицательных значений days_employed: 15906
Количество положительных значений days_employed: 3445


Приблизительно 74% в столбце "days_employed" - отрицательные. Приведем весь столбец к положительным значениям, применив функцию abs():

In [21]:
borrowers['days_employed'] = borrowers['days_employed'].abs()
borrowers[borrowers['days_employed']<0].shape[0]


0


Сгруппируем нашу таблицу по столбцу "enployment_type" для каждого из столбцов "days_employed" и "salary" и заменим NaN в этих столбцах на среднее значение по каждой группировке

In [22]:
borrowers['days_employed'] = borrowers.groupby('enployment_type')['days_employed'].transform(lambda x: x.fillna(x.mean()))
borrowers['salary'] = borrowers.groupby('enployment_type')['salary'].transform(lambda x: x.fillna(x.mean()))

borrowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
age                 21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
enployment_type     21525 non-null object
credit_debt         21525 non-null int64
salary              21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Также обработаем значения в столбце "age". Сгруппируем данные "enployment_type" для столбца "age" и заменим нули в этом столбцех на среднее значение, исходя из типа занятости клиента:

In [23]:
borrowers['age'] = borrowers.groupby('enployment_type')['age'].transform(lambda x: x.replace(0,int(x.mean())))

In [24]:
borrowers.describe()

,children,days_employed,age,education_id,family_status_id,credit_debt,salary
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.543275,67352.246350,43.495238,0.817236,0.972544,0.080883,1.673959e+05
std,1.379876,139366.594574,12.230322,0.548138,1.420324,0.272661,9.790695e+04
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.549623,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2326.499216,43.000000,1.000000,0.000000,0.000000,1.519313e+05
75%,1.000000,5347.024506,53.000000,1.000000,1.000000,0.000000,2.024175e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [25]:
borrowers['days_employed'] = borrowers['days_employed'].astype(int)
borrowers['salary'] = borrowers['salary'].astype(int)
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
age                 21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
enployment_type     21525 non-null object
credit_debt         21525 non-null int64
salary              21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [26]:
borrowers.duplicated().sum()

54

In [27]:
borrowers[borrowers.duplicated(keep=False)].sort_values(by = ['days_employed','salary','purpose'])

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,enployment_type,credit_debt,salary,purpose
9374,0,2111,38,высшее,0,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
19387,0,2111,38,высшее,0,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
2254,0,2111,54,высшее,0,женат / замужем,0,M,компаньон,0,202417,операции с коммерческой недвижимостью
17379,0,2111,54,высшее,0,женат / замужем,0,M,компаньон,0,202417,операции с коммерческой недвижимостью
8490,1,2111,31,среднее,1,женат / замужем,0,F,компаньон,0,202417,покупка жилья
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0,365003,57,среднее,1,гражданский брак,1,F,пенсионер,0,137127,свадьба
15273,0,365003,57,среднее,1,гражданский брак,1,F,пенсионер,0,137127,свадьба
20116,0,365003,57,среднее,1,гражданский брак,1,M,пенсионер,0,137127,свадьба
2052,0,365003,58,среднее,1,гражданский брак,1,F,пенсионер,0,137127,сыграть свадьбу


In [28]:
borrowers = borrowers.drop_duplicates()
borrowers.duplicated().sum()

0

In [29]:
borrowers['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [30]:
borrowers['education'] = borrowers['education'].str.lower()
borrowers['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

### Лемматизация

In [31]:
borrowers['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [32]:
m = Mystem()
list_of_unique_lemmas = []

for element in borrowers['purpose']:
    lemmas = m.lemmatize(element)
    list_of_unique_lemmas.extend(lemmas)
    
counter_lemmas = Counter(list_of_unique_lemmas)

#Выведем общий список лемм, отсортировав его по количественному убыванию, чтобы понять, какие слова используются чаще всего. 
#Так будет удобнее определить основые цели по взятию кредита.

data_of_counter_lemmas = sorted(counter_lemmas.items(), key=lambda x: x[1], reverse = True)
data_of_counter_lemmas

[(' ', 33596),
 ('\n', 21471),
 ('недвижимость', 6353),
 ('покупка', 5900),
 ('жилье', 4461),
 ('автомобиль', 4308),
 ('образование', 4014),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2335),
 ('свой', 2231),
 ('на', 2228),
 ('строительство', 1879),
 ('высокий', 1374),
 ('получение', 1315),
 ('коммерческий', 1312),
 ('для', 1290),
 ('жилой', 1231),
 ('сделка', 941),
 ('дополнительный', 907),
 ('заниматься', 904),
 ('проведение', 773),
 ('сыграть', 769),
 ('сдача', 652),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

In [33]:
list_of_purpose = ['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба']
list_of_purpose

['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба']

### Категоризация данных

In [34]:
russian_stemmer = SnowballStemmer('russian')
stem_list_of_purpose = []
for word in list_of_purpose:
    stem_list_of_purpose.append(russian_stemmer.stem(word))
print(stem_list_of_purpose)

['недвижим', 'жил', 'автомобил', 'образован', 'свадьб']


In [35]:
dict_of_purpose = {val:key for key, val in enumerate(stem_list_of_purpose)}
dict_of_purpose

{'недвижим': 0, 'жил': 1, 'автомобил': 2, 'образован': 3, 'свадьб': 4}

In [36]:
dict_of_purpose['недвижим'] = 1
dict_of_purpose

{'недвижим': 1, 'жил': 1, 'автомобил': 2, 'образован': 3, 'свадьб': 4}

Чтобы определить категории для столбца purpose создадим функцию purpose_index(), которая будет проходить по ключам словаря dict_of_purpose и проверять, находится ли ключ из этого словаря  в значении из столбца purpose в нашем датафрейме. 
Функция будет возвращать номер из словаря, который соответсвует ключу, совпавшему со значением в столбце purpose.

In [37]:
def purpose_index(row):
    purpose = row['purpose']
    for word in dict_of_purpose.keys():
        if word in purpose:
            return dict_of_purpose[word]

In [38]:
borrowers['purpose_index'] = borrowers.apply(purpose_index, axis = 1)
borrowers.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,enployment_type,credit_debt,salary,purpose,purpose_index
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4


Далее необходимо разбить на категории размер зарплаты каждого клиента (столбец "salary"). Я брала примерные значения, ориентируюясь на данные опроса Россиян СК "Россгосстрах жизнь" совместно с центром "Перспектива" за 2020 год. 

Ссылка - https://www.rbc.ru/economics/07/07/2020/5eff14959a794725e351de43 .

И так как в наших данных присутствуют значения с зарплатой больше миллиона, определим это понятие как "очень богатый"

- бедный : до 25 тыс. рублей 
- средний : 25 - 50 тыс. рублей
- выше среднего : 50 - 100 тыс. рублей
- богатый : от 100 тыс. рублей до 1 млн. рублей
- очень богатый : более 1 млн. рублей

In [39]:
def salary_group(row):
    if row['salary'] <= 25000:
        return 'бедный'
    elif  25000 < row['salary'] <= 50000:
        return 'средний'
    elif  50000 < row['salary'] <= 100000:
        return 'выше среднего'
    elif 100000 < row['salary'] <= 1000000:
        return 'богатый'
    else:
        return 'очень богатый'
    
borrowers['salary_group'] = borrowers.apply(salary_group, axis = 1)
borrowers.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,enployment_type,credit_debt,salary,purpose,purpose_index,salary_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,богатый
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2,богатый
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,богатый
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3,богатый
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4,богатый


In [40]:
borrowers[borrowers['children']==20]['children'].count()

76

имеем 76 значений. Мое предположение - ноль сюда попал случайно. Можно данные из 20 исправить на 2.

In [41]:
borrowers['children'] = borrowers['children'].replace(20,2)
borrowers[borrowers['children']==20]['children'].count()

0

Теперь можно создать функцию children_group(), которая будет разбивать клиентов в зависимости от количества имеющихся детей на следующие категории. 

- 'нет детей'
- '1-2 ребенка'
- '3 ребенка и более'

После этого сформируем новый столбец "children_group", применив эту функцию к датафрейму:

In [42]:
def children_group(row):
    if row['children'] == 0:
        return 'нет детей'
    elif row['children'] == 1 or row['children'] == 2:
        return '1-2 ребенка'
    else:
        return '3 ребенка и более'
    
borrowers['children_group'] = borrowers.apply(children_group, axis = 1)
borrowers.head()

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,enployment_type,credit_debt,salary,purpose,purpose_index,salary_group,children_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,1,богатый,1-2 ребенка
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,2,богатый,1-2 ребенка
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,1,богатый,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3,богатый,3 ребенка и более
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,4,богатый,нет детей


## Вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [48]:
borrowers.groupby('children_group')[['credit_debt']].mean().sort_values('credit_debt').style.format({'credit_debt': '{:.2%}'})

,credit_debt
children_group,
нет детей,7.54%
3 ребенка и более,8.16%
1-2 ребенка,9.26%


В среднем клиенты, у которых нет детей менее склонны к задолженности по кредиту

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [44]:
borrowers.groupby('family_status')['credit_debt'].mean().to_frame().sort_values('credit_debt')

,credit_debt
family_status,
вдовец / вдова,0.065693
в разводе,0.071130
женат / замужем,0.075421
гражданский брак,0.093202
Не женат / не замужем,0.097509


Клиенты, которые не вступили в законный брак более склонны к задолженности по кредиту

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [45]:
borrowers.groupby('salary_group')['credit_debt'].mean().to_frame().sort_values('credit_debt')

,credit_debt
salary_group,
средний,0.060440
очень богатый,0.080000
выше среднего,0.080909
богатый,0.081552
бедный,0.125000


Клиенты, уровень дохода у которых очень низкий  - склонны к задолженностям по выплате кредита. 

Но если отталкиваться уже от среднего уровня дохода и выше, то можно заметить, что чем больше получает клиент, тем более он склонен к возникновению задолженности

- Как разные цели кредита влияют на его возврат в срок?

In [46]:
borrowers.groupby('purpose_index')['credit_debt'].mean().to_frame().sort_values('credit_debt')

,credit_debt
purpose_index,
1,0.072314
4,0.079657
3,0.092177
2,0.093547


Напомним, какой индекс - чему соответсвует:

- 1 - операции с покупкой жилья/недвижимости
- 2 - приобретение автомобиля
- 3 - оплата образования
- 4 - проведение свадьбы

Таким образом можно сделать вывод, что клиенты, оформляющие кредит на оплату за образование или на покупку авто - более склонны к задолженности, чем те, кто планирует приобрести жилье или сыграть свадьбу.

## Общий вывод

В ходе работы мы провели всю необходимую обработку данных, а именно: заполнили пропуски и обработали странные значения в данных, удалили дубликаты, привели количественные значения к нужному типу, провели лемматизацию и категоризацию данных. 

Исходя из проделанной работы и проведения анализа можно сделать общий вывод о том, как влияет семейное положение и количество детей клиента на факт погашения кредита в срок:

- Клиенты, которые уже вступили в законный брак, имеющие средний уровень дохода, но не имеющие детей - предрасположены к своевременной выплате по кредиту. 
- Клиенты, у которых есть 1-2 ребенка, но нет узаконенных отношений - более склонны к задолженности.